In [1]:
import pandas as pd
import math
import scipy.stats as st
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [36]:
df = pd.read_csv("C:/Users/eyp_d/Desktop/VBO-Bootcamp-Dersler/5.Hafta/Ders Öncesi Notlar/amazon_review.csv")

In [37]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0


In [38]:
df["asin"].nunique()

1

## Görev 1:

#### Average Rating’i güncel yorumlara göre hesaplayınız ve var olan average rating ile kıyaslayınız.

In [39]:
df["overall"].mean() # Ürünün var olan ratingi: https://www.amazon.com/SanDisk-Ultra-MicroSDXC-Memory-Adapter/dp/B007WTAJTO

4.587589013224822

In [40]:
quantiles = df["day_diff"].quantile([0.1, 0.25, 0.50, 0.75]).values.tolist()

In [41]:
def time_based_weighted_average(dataframe, w1=25, w2=23, w3=20, w4=17, w5=15):   
    return df.loc[df["day_diff"] <= quantiles[0], "overall"].mean() * w1 / 100 + \
           df.loc[(df["day_diff"] > quantiles[0]) & (df["day_diff"] <= quantiles[1]), "overall"].mean() * w2 / 100 + \
           df.loc[(df["day_diff"] > quantiles[1]) & (df["day_diff"] <= quantiles[2]), "overall"].mean() * w3 / 100 + \
           df.loc[(df["day_diff"] > quantiles[2]) & (df["day_diff"] <= quantiles[3]), "overall"].mean() * w4 / 100 + \
           df.loc[(df["day_diff"] > quantiles[3]), "overall"].mean() * w5 / 100
    

In [42]:
time_based_weighted_average(df)  # Time-based weighted rating 

4.6247824795763

## Görev 2:

#### Ürün için ürün detay sayfasında görüntülenecek 20 review’i belirleyiniz.

In [43]:
df_review = df[["reviewerID", "helpful_yes", "total_vote"]]
df_review.sort_values("total_vote", ascending=False).head()

,reviewerID,helpful_yes,total_vote
2031,A12B7ZMXFI6IXY,1952,2020
4212,AVBMZZAFEKO58,1568,1694
3449,AOEAD7DPLZE53,1428,1505
317,A1ZQAQFYSXL5MQ,422,495
2909,A10B6G6W3DW1EY,53,236


In [44]:
df_review = df_review.loc[~(df_review["total_vote"]==0)]  # Yorumuna hiç vote yapılmayanlar çıkarıldı.
df_review.head()

,reviewerID,helpful_yes,total_vote
8,A2CYJO155QP33S,1,1
17,A1OATGH4DF0RF0,0,1
26,A1U9QJQW3XPRL2,1,2
28,A2U4VW7FAB7W9A,1,1
37,AKVRSGKPY2C4R,1,1


In [45]:
df_review["helpful_no"] = df_review["total_vote"] - df_review["helpful_yes"]
df_review.head()

,reviewerID,helpful_yes,total_vote,helpful_no
8,A2CYJO155QP33S,1,1,0
17,A1OATGH4DF0RF0,0,1,1
26,A1U9QJQW3XPRL2,1,2,1
28,A2U4VW7FAB7W9A,1,1,0
37,AKVRSGKPY2C4R,1,1,0


In [46]:
df_review = df_review.reindex(columns=["reviewerID", "total_vote", "helpful_yes", "helpful_no"])  # Verinin daha iyi okunabilmesi için
df_review.head()

,reviewerID,total_vote,helpful_yes,helpful_no
8,A2CYJO155QP33S,1,1,0
17,A1OATGH4DF0RF0,1,0,1
26,A1U9QJQW3XPRL2,2,1,1
28,A2U4VW7FAB7W9A,1,1,0
37,AKVRSGKPY2C4R,1,1,0


In [47]:
def wilson_lower_bound(up, down, confidence=0.95):
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

In [48]:
df_review["wilson_lower_bound"] = df_review.apply(lambda x: wilson_lower_bound(x["helpful_yes"], x["helpful_no"]),axis=1)
df_review.sort_values("wilson_lower_bound", ascending=False).head()

,reviewerID,total_vote,helpful_yes,helpful_no,wilson_lower_bound
2031,A12B7ZMXFI6IXY,2020,1952,68,0.95754
3449,AOEAD7DPLZE53,1505,1428,77,0.93652
4212,AVBMZZAFEKO58,1694,1568,126,0.91214
317,A1ZQAQFYSXL5MQ,495,422,73,0.81858
4672,A2DKQQIZ793AV5,49,45,4,0.80811


In [49]:
top_reviewerID_list = df_review.sort_values("wilson_lower_bound", ascending=False)["reviewerID"].head(20).values.tolist()
top_reviewerID_list

['A12B7ZMXFI6IXY',
 'AOEAD7DPLZE53',
 'AVBMZZAFEKO58',
 'A1ZQAQFYSXL5MQ',
 'A2DKQQIZ793AV5',
 'A1J6VSUM80UAF8',
 'A1K91XXQ6ZEBQR',
 'AFGRMORWY2QNX',
 'AOHXKM5URSKAB',
 'A1WTQUOQ4WG9AI',
 'A2J26NNQX6WKAU',
 'A2TPXOZSU1DACQ',
 'A6I8KXYK24RTB',
 'A2EL2GWJ9T0DWY',
 'A22GOZTFA02O2F',
 'A2Z4VVF1NTJWPB',
 'ALPLKR59QMBUX',
 'A3MEPYZVTAV90W',
 'A1PLHPPAJ5MUXG',
 'A2O96COBMVY9C4']

In [58]:
df.set_index('reviewerID').loc[top_reviewerID_list].reset_index()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1367366400,2013-01-05,702,1952,2020
1,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.00000,Top of the class among all (budget-priced) mic...,1348617600,2012-09-26,803,1428,1505
2,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1375660800,2013-05-08,579,1568,1694
3,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.00000,"Warning, read this!",1346544000,2012-02-09,1033,422,495
4,A2DKQQIZ793AV5,B007WTAJTO,Twister,"[45, 49]",Sandisk announcement of the first 128GB micro ...,5.00000,Super high capacity!!! Excellent price (on Am...,1394150400,2014-07-03,158,45,49
5,A1J6VSUM80UAF8,B007WTAJTO,goconfigure,"[60, 68]",Bought from BestBuy online the day it was anno...,5.00000,I own it,1393545600,2014-02-28,283,60,68
6,A1K91XXQ6ZEBQR,B007WTAJTO,"R. Sutton, Jr. ""RWSynergy""","[112, 139]",The last few days I have been diligently shopp...,5.00000,"Resolving confusion between ""Mobile Ultra"" and...",1350864000,2012-10-22,777,112,139
7,AFGRMORWY2QNX,B007WTAJTO,R. Heisler,"[22, 25]",I bought this card to replace a lost 16 gig in...,3.00000,"Good buy for the money but wait, I had an issue!",1361923200,2013-02-27,649,22,25
8,AOHXKM5URSKAB,B007WTAJTO,Stellar Eller,"[51, 65]","While I got this card as a ""deal of the day"" o...",5.00000,Awesome Card!,1339200000,2012-09-06,823,51,65
9,A1WTQUOQ4WG9AI,B007WTAJTO,"Tom Henriksen ""Doggy Diner""","[82, 109]",Hi:I ordered two card and they arrived the nex...,1.00000,Designed incompatibility/Don't support SanDisk,1348272000,2012-09-22,807,82,109
